In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Task 1

In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/IBM_GRM_Project/compliance_data.xls'
df = pd.read_excel(file_path)
df


,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16T11:34:07Z,NaN,NaN,NaN,1. duplication of one course completed: return...
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T13:03:52Z,NaN,NaN,NaN,2. flag students identify disposable mail (int...
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T13:45:44Z,NaN,NaN,NaN,3. flag courses and student ids completion in ...
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T14:00:09Z,NaN,NaN,NaN,4. numbers of unique courses completed per day...
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T14:24:03Z,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378,Getting Started with Artificial Intelligence,PLAN-E624C2604060,0.0,Learning Plan,Your Learning Builder - Plans,22WJ8A1277 KATIPELLY UDAY REDDY,NaN,3671307REG,In Progress,NaN,NaN,NaN,NaN,NaN
16379,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:31:11Z,NaN,NaN,NaN,NaN
16380,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:38:16Z,NaN,NaN,NaN,NaN
16381,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:35:57Z,NaN,NaN,NaN,NaN


In [ ]:
# Filter only completed courses
completed_df = df[df['Transcript status'] == 'Completed']


dupes = completed_df[completed_df.duplicated(subset=['Learner - ID', 'Learning activity - Title'], keep=False)]
dupes

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
3531,Use Generative AI for Software Development,MDL-504,60.0,eLearning,Moodle,Pranitha Naroju,NaN,3452637REG,Completed,2024-12-01T15:04:09Z,NaN,NaN,NaN,NaN
3532,Use Generative AI for Software Development,MDL-504,60.0,eLearning,Moodle,Pranitha Naroju,NaN,3452637REG,Completed,2024-10-28T11:23:53Z,NaN,NaN,NaN,NaN
4389,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Vedhanarayanan .K Kannan Family,NaN,3571326REG,Completed,2024-12-02T07:12:36Z,NaN,NaN,NaN,NaN
4390,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Vedhanarayanan .K Kannan Family,NaN,3571326REG,Completed,2024-11-13T00:24:39Z,NaN,NaN,NaN,NaN
4559,Journey to Cloud: Envisioning Your Solution,MDL-447,300.0,eLearning,Moodle,Vaishnavi R,NaN,3575765REG,Completed,2024-11-08T01:18:51Z,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13656,Journey to Cloud: Envisioning Your Solution,PLAN-32CB1E21D8B4,0.0,Learning Plan,Your Learning Builder - Plans,Sai srinu parasa,NaN,3630363REG,Completed,2024-11-23T08:15:58.787680Z,NaN,NaN,NaN,NaN
13896,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Kalapala Manogna,NaN,3632705REG,Completed,2024-11-20T10:10:25Z,NaN,NaN,NaN,NaN
13897,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Kalapala Manogna,NaN,3632705REG,Completed,2024-11-29T06:34:34Z,NaN,NaN,NaN,NaN
14428,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Sailash S,NaN,3642066REG,Completed,2024-11-22T05:58:35Z,NaN,NaN,NaN,NaN


In [ ]:
df1=df

In [ ]:
df1

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16T11:34:07Z,NaN,NaN,NaN,1. duplication of one course completed: return...
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T13:03:52Z,NaN,NaN,NaN,2. flag students identify disposable mail (int...
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T13:45:44Z,NaN,NaN,NaN,3. flag courses and student ids completion in ...
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T14:00:09Z,NaN,NaN,NaN,4. numbers of unique courses completed per day...
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T14:24:03Z,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378,Getting Started with Artificial Intelligence,PLAN-E624C2604060,0.0,Learning Plan,Your Learning Builder - Plans,22WJ8A1277 KATIPELLY UDAY REDDY,NaN,3671307REG,In Progress,NaN,NaN,NaN,NaN,NaN
16379,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:31:11Z,NaN,NaN,NaN,NaN
16380,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:38:16Z,NaN,NaN,NaN,NaN
16381,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:35:57Z,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(16383, 14)

In [ ]:
print(df1['Completion Date'].dtype)


object


In [ ]:
empty_count = df1['Completion Date'].isna().sum()
print(f"Number of empty or missing values in Completion Date: {empty_count}")


Number of empty or missing values in Completion Date: 2917


In [ ]:
df1 = df1.dropna(subset=['Completion Date'])

In [ ]:
df1.shape

(13466, 14)

In [ ]:
import pandas as pd
import pytz  # For timezone conversion

# Step 1: Parse 'Completion Date' as datetime with timezone (UTC)
df1['Completion Date'] = pd.to_datetime(df1['Completion Date'], utc=True, errors='coerce')

/tmp/ipython-input-13-1717338698.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Completion Date'] = pd.to_datetime(df1['Completion Date'], utc=True, errors='coerce')


In [ ]:
df1['Completion Date'].shape

(13466,)

In [ ]:
ist = pytz.timezone('Asia/Kolkata')
df1['Completion Date'] = df1['Completion Date'].dt.tz_convert(ist)

/tmp/ipython-input-15-3331913316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Completion Date'] = df1['Completion Date'].dt.tz_convert(ist)


In [ ]:
df1['Completion Date']

,Completion Date
0,2024-08-16 17:04:07+05:30
1,2024-08-28 18:33:52+05:30
2,2024-08-28 19:15:44+05:30
3,2024-08-28 19:30:09+05:30
4,2024-08-28 19:54:03+05:30
...,...
16367,NaT
16368,NaT
16379,2024-11-30 11:01:11+05:30
16380,2024-11-30 11:08:16+05:30


In [ ]:
df1_sorted = df1.sort_values(by='Completion Date', ascending=False)

In [ ]:
df1_sorted['Completion Date']

,Completion Date
12195,2025-01-30 10:07:07+05:30
8945,2025-01-30 00:10:39+05:30
8674,2025-01-29 20:37:55+05:30
8675,2025-01-29 20:17:19+05:30
8052,2025-01-29 20:07:09+05:30
...,...
16351,NaT
16356,NaT
16366,NaT
16367,NaT


In [ ]:
duplicates = df1_sorted[df1_sorted.duplicated(subset=['Learner - ID', 'Learning activity - Title'], keep=False)]

### Table containing the duplicate values of the whole dataset

In [ ]:
duplicates[['Learner - ID', 'Learner - Name', 'Learning activity - Title', 'Learning activity - ID','Completion Date']]

,Learner - ID,Learner - Name,Learning activity - Title,Learning activity - ID,Completion Date
8053,3596831REG,Naveen Prithivraj,Use Generative AI for Software Development,MDL-504,2025-01-29 19:52:22+05:30
5286,3582071REG,Dharshini Priya,Build Your First Chatbot,MDL-510,2025-01-29 11:25:59+05:30
10333,3603350REG,Pradhoshika D,Use Generative AI for Software Development,MDL-504,2025-01-23 15:14:43+05:30
10305,3603345REG,R. Durga Devi,Use Generative AI for Software Development,MDL-504,2025-01-23 15:05:58+05:30
7952,3596768REG,Prasanna Balaji C,Use Generative AI for Software Development,MDL-504,2025-01-21 19:56:08+05:30
...,...,...,...,...,...
5945,3586247REG,K.Sri Mahalakshmi,Journey to Cloud: Envisioning Your Solution,PLAN-32CB1E21D8B4,NaT
11309,3611981REG,Elageshwaran S,Summary (A Taste of Agile),ILB-PZDGJGDZWZYQ278K,NaT
11320,3611981REG,Elageshwaran S,Summary (A Taste of Agile),ILB-ZKVWJWVDDKGM37RN,NaT
13606,3630339REG,NAGA VENKATA TEJA GUDAVALLI,Journey to Cloud: Envisioning Your Solution,PLAN-32CB1E21D8B4,NaT


to do:duplication of courses repeated info

In [ ]:
latest_duplicates = df1_sorted.drop_duplicates(
    subset=['Learner - ID', 'Learning activity - Title'],
    keep='first'
)

In [ ]:
latest_duplicates[['Learner - ID', 'Learner - Name', 'Learning activity - Title', 'Learning activity - ID','Latest Completion Date']]


KeyError: "['Latest Completion Date'] not in index"

In [ ]:
final_df = latest_duplicates[[
    'Learner - ID',
    'Learner - Name',
    'Learning activity - Title',
    'Learning activity - ID',
    'Completion Date'
]]


In [ ]:
final_df

In [ ]:
duplicates = final_df.duplicated(subset=['Learner - ID', 'Learning activity - Title']).sum()
print(f"Duplicates left: {duplicates}")


...............................................................end..............................................................

In [ ]:
completion_date_idx = df1.columns.get_loc('Completion Date')

cols_to_keep = df1.columns[:completion_date_idx + 1]

# Create a new DataFrame with only those columns
df_trimmed = df1[cols_to_keep]
df_trimmed.shape

In [ ]:
# Rows where 'Completion Date' is NaT
nat_rows = df1[df1['Completion Date'].isna()]

print("Rows with unparseable or missing Completion Dates after conversion:")
nat_rows

In [ ]:
print(repr(df.loc[1, 'Completion Date']))


========================================================================================================

## Task 4: numbers of unique courses completed per day per student should not exceed 50:
## return students < 50 and > 50 courses per per student day (table)

In [23]:
end_col = df.columns.get_loc('Completion Date')

df2 = df.iloc[:, :end_col + 1]

df2 = df2.dropna(subset=['Completion Date'])

In [24]:
df2

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16T11:34:07Z
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T13:03:52Z
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T13:45:44Z
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T14:00:09Z
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28T14:24:03Z
...,...,...,...,...,...,...,...,...,...,...
16367,Create a Credly account,URL-0E39749E2965,6.0,Guidance,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-12-01T05:50:00.508255Z
16368,Earn it! Accept it! Share it! | IBM SkillsBuild,URL-CC7432BB7A8A,2.0,Video,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-11-28T17:34:21.389938Z
16379,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:31:11Z
16380,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30T05:38:16Z


In [25]:
df2.columns

Index(['Learning activity - Title', 'Learning activity - ID',
       'Learning activity - Duration', 'Delivery Type', 'Learning Source Name',
       'Learner - Name', 'Learner - Email', 'Learner - ID',
       'Transcript status', 'Completion Date'],
      dtype='object')

In [26]:
THRESHOLD = 5

df2['Completion Date'] = pd.to_datetime(df2['Completion Date'], errors='coerce')

df2['Completion Day'] = df2['Completion Date'].dt.date

course_counts = (
    df2.groupby(['Learner - ID', 'Completion Day'])['Learning activity - ID']
    .nunique()
    .reset_index(name='courses_completed')
)

df_merged = df2.merge(course_counts, on=['Learner - ID', 'Completion Day'])

df_less_than_threshold = df_merged[df_merged['courses_completed'] < THRESHOLD]
df_more_than_threshold = df_merged[df_merged['courses_completed'] >= THRESHOLD]

df_less_than_threshold = df_less_than_threshold.drop(columns=['courses_completed', 'Completion Day'])
df_more_than_threshold = df_more_than_threshold.drop(columns=['courses_completed', 'Completion Day'])

In [27]:
print(f"Students with < {THRESHOLD} courses per day: {df_less_than_threshold.shape[0]}")

print(f"Students with >= {THRESHOLD} courses per day: {df_more_than_threshold.shape[0]}")


Students with < 5 courses per day: 7474
Students with >= 5 courses per day: 605


In [28]:
df_less_than_threshold

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:34:07+00:00
7,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:49:56+00:00
8,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:45:14+00:00
9,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Ayesha Maryam,ayxxxx@gmail.com,3088998REG,Completed,2024-08-17 08:02:54+00:00
10,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ayesha Maryam,ayxxxx@gmail.com,3088998REG,Completed,2024-08-16 13:18:10+00:00
...,...,...,...,...,...,...,...,...,...,...
8074,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-12-09 15:01:03+00:00
8075,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-11-29 18:15:36+00:00
8076,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30 05:31:11+00:00
8077,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30 05:38:16+00:00


In [29]:
df_more_than_threshold

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:03:52+00:00
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:45:44+00:00
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:00:09+00:00
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:24:03+00:00
5,Make an Impact with Hybrid Cloud,MDL-291,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:49:51+00:00
...,...,...,...,...,...,...,...,...,...,...
7733,Your Future in AI: The Job Landscape,MDL-213,50.0,eLearning,Moodle,NIKHITHA T A ECE,NaN,3657266REG,Completed,2024-11-27 13:35:35+00:00
7734,AI Ethics,MDL-220,105.0,eLearning,Moodle,NIKHITHA T A ECE,NaN,3657266REG,Completed,2024-11-27 13:08:07+00:00
7735,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,NIKHITHA T A ECE,NaN,3657266REG,Completed,2024-11-27 11:58:52+00:00
7736,The Money Maker GANs Game,MDL-317,10.0,eLearning,Moodle,NIKHITHA T A ECE,NaN,3657266REG,Completed,2024-11-27 12:04:45+00:00


In [30]:
# Grouping for better readability: Student-wise, per day, list their completed courses
def summarize_by_student(df2):
    summary = (
        df2.groupby(['Learner - ID', 'Learner - Name', 'Completion Date'])
        .agg({
            'Learning activity - Title': lambda x: list(x),
            'Learning activity - ID': lambda x: list(x),
        })
        .reset_index()
        .rename(columns={
            'Learning activity - Title': 'Courses Completed (Titles)',
            'Learning activity - ID': 'Courses Completed (IDs)'
        })
    )
    return summary

# Create readable summaries
summary_less_than_threshold = summarize_by_student(df_less_than_threshold)
summary_more_than_threshold = summarize_by_student(df_more_than_threshold)


In [31]:
summary_less_than_threshold.head(25)

,Learner - ID,Learner - Name,Completion Date,Courses Completed (Titles),Courses Completed (IDs)
0,3088627REG,Adit Dhiman,2024-08-16 11:34:07+00:00,[Introduction to Artificial Intelligence],[MDL-211]
1,3088627REG,Adit Dhiman,2024-08-16 11:45:14+00:00,[Large Language Model Basics],[MDL-433]
2,3088627REG,Adit Dhiman,2024-08-16 11:49:56+00:00,[Mastering the Art of Prompting],[MDL-298]
3,3088998REG,Ayesha Maryam,2024-08-16 13:18:10+00:00,[Mastering the Art of Prompting],[MDL-298]
4,3088998REG,Ayesha Maryam,2024-08-17 08:02:54+00:00,[Introduction to Artificial Intelligence],[MDL-211]
5,3088998REG,Ayesha Maryam,2024-08-17 08:11:53+00:00,[Large Language Model Basics],[MDL-433]
6,3089312REG,Kabilan M,2024-08-16 17:06:07+00:00,[Mastering the Art of Prompting],[MDL-298]
7,3089312REG,Kabilan M,2024-08-16 17:37:22+00:00,[Introduction to Artificial Intelligence],[MDL-211]
8,3089312REG,Kabilan M,2024-08-17 03:21:35+00:00,[Large Language Model Basics],[MDL-433]
9,3091818REG,PARAMATA MANO ADITYA 236N1A0486,2024-08-18 10:51:33+00:00,[Introduction to Artificial Intelligence],[MDL-211]


In [32]:
summary_more_than_threshold.head(25)

,Learner - ID,Learner - Name,Completion Date,Courses Completed (Titles),Courses Completed (IDs)
0,3088627REG,Adit Dhiman,2024-08-28 13:03:52+00:00,"[Climate, Biodiversity, and Human Society (bro...",[MDL-285]
1,3088627REG,Adit Dhiman,2024-08-28 13:45:44+00:00,[Beyond Conservation to Sustainability],[MDL-288]
2,3088627REG,Adit Dhiman,2024-08-28 14:00:09+00:00,[Make an Impact with Data Analytics],[MDL-289]
3,3088627REG,Adit Dhiman,2024-08-28 14:24:03+00:00,[Make an Impact with AI],[MDL-290]
4,3088627REG,Adit Dhiman,2024-08-28 14:49:51+00:00,[Make an Impact with Hybrid Cloud],[MDL-291]
5,3088627REG,Adit Dhiman,2024-08-28 15:01:14+00:00,[Make an Impact with Future Technology],[MDL-292]
6,3092060REG,Chandra Bhanu Koppadi,2024-09-24 16:05:31+00:00,[Beyond Conservation to Sustainability],[MDL-288]
7,3092060REG,Chandra Bhanu Koppadi,2024-09-24 16:24:29+00:00,[Make an Impact with Data Analytics],[MDL-289]
8,3092060REG,Chandra Bhanu Koppadi,2024-09-24 16:42:34+00:00,[Make an Impact with AI],[MDL-290]
9,3092060REG,Chandra Bhanu Koppadi,2024-09-24 17:07:15+00:00,[Make an Impact with Hybrid Cloud],[MDL-291]


## Task 2: flag students identify disposable mail

### Method 1: Manual insertions

In [33]:
df3 = df2
df3

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Completion Day
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:34:07+00:00,2024-08-16
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:03:52+00:00,2024-08-28
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:45:44+00:00,2024-08-28
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:00:09+00:00,2024-08-28
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:24:03+00:00,2024-08-28
...,...,...,...,...,...,...,...,...,...,...,...
16367,Create a Credly account,URL-0E39749E2965,6.0,Guidance,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT
16368,Earn it! Accept it! Share it! | IBM SkillsBuild,URL-CC7432BB7A8A,2.0,Video,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT
16379,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30 05:31:11+00:00,2024-11-30
16380,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30 05:38:16+00:00,2024-11-30


In [34]:
import random

# Sample real and disposable domains
genuine_domains = ['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'icloud.com']
disposable_domains = ['mailinator.com', '10minutemail.com', 'tempmail.com', 'guerrillamail.com', 'trashmail.com']

# Assign random emails to each row (assuming df is your DataFrame)
def generate_dummy_email(name):
    domain = random.choice(genuine_domains + disposable_domains)
    username = name.lower().replace(" ", ".")
    return f"{username}@{domain}"

df3['DummyEmails'] = df3['Learner - Name'].apply(generate_dummy_email)


In [35]:
df3['DummyEmails']

,DummyEmails
0,adit.dhiman@trashmail.com
1,adit.dhiman@guerrillamail.com
2,adit.dhiman@outlook.com
3,adit.dhiman@hotmail.com
4,adit.dhiman@gmail.com
...,...
16367,mamidimada.srikanth@hotmail.com
16368,mamidimada.srikanth@gmail.com
16379,ekalavya.sunvith.chichadi@mailinator.com
16380,ekalavya.sunvith.chichadi@hotmail.com


In [36]:
# List of common genuine domains (can be expanded)
common_domains = {'gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'icloud.com'}

# Function to detect disposable domains
def is_disposable(email):
    try:
        domain = email.split('@')[1].lower()
        return domain not in common_domains
    except:
        return True  # Consider malformed emails as disposable

# Apply detection
df3['IsDisposable'] = df3['DummyEmails'].apply(is_disposable)


In [37]:
disposable_emails_df = df3[df3['IsDisposable']]
genuine_emails_df = df3[~df3['IsDisposable']]


In [38]:
disposable_emails_df

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Completion Day,DummyEmails,IsDisposable
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:34:07+00:00,2024-08-16,adit.dhiman@trashmail.com,True
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:03:52+00:00,2024-08-28,adit.dhiman@guerrillamail.com,True
6,Make an Impact with Future Technology,MDL-292,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 15:01:14+00:00,2024-08-28,adit.dhiman@guerrillamail.com,True
16,Create a Credly account,URL-0E39749E2965,20.0,Guidance,Your Learning Builder - Activities,Harsh Kumar,haxxxx@gmail.com,3088744REG,Completed,NaT,NaT,harsh.kumar@tempmail.com,True
19,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ayesha Maryam,ayxxxx@gmail.com,3088998REG,Completed,2024-08-16 13:18:10+00:00,2024-08-16,ayesha.maryam@mailinator.com,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16356,Create a Credly account,URL-0E39749E2965,5.0,Guidance,Your Learning Builder - Activities,Vanama mansi Chowdary,NaN,3671281REG,Completed,NaT,NaT,vanama.mansi.chowdary@guerrillamail.com,True
16361,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-12-08 12:22:44+00:00,2024-12-08,mamidimada.srikanth@tempmail.com,True
16364,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-11-29 18:15:36+00:00,2024-11-29,mamidimada.srikanth@trashmail.com,True
16366,Getting Started with Artificial Intelligence,PLAN-E624C2604060,0.0,Learning Plan,Your Learning Builder - Plans,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT,mamidimada.srikanth@mailinator.com,True


In [39]:
genuine_emails_df

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Completion Day,DummyEmails,IsDisposable
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:45:44+00:00,2024-08-28,adit.dhiman@outlook.com,False
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:00:09+00:00,2024-08-28,adit.dhiman@hotmail.com,False
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:24:03+00:00,2024-08-28,adit.dhiman@gmail.com,False
5,Make an Impact with Hybrid Cloud,MDL-291,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:49:51+00:00,2024-08-28,adit.dhiman@gmail.com,False
7,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:49:56+00:00,2024-08-16,adit.dhiman@hotmail.com,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16363,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-12-09 15:01:03+00:00,2024-12-09,mamidimada.srikanth@hotmail.com,False
16367,Create a Credly account,URL-0E39749E2965,6.0,Guidance,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT,mamidimada.srikanth@hotmail.com,False
16368,Earn it! Accept it! Share it! | IBM SkillsBuild,URL-CC7432BB7A8A,2.0,Video,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT,mamidimada.srikanth@gmail.com,False
16380,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30 05:38:16+00:00,2024-11-30,ekalavya.sunvith.chichadi@hotmail.com,False


### Method 2: Using Disposable Domain List

Git hub repo: https://github.com/disposable-email-domains/disposable-email-domains

Disposable List: https://github.com/disposable-email-domains/disposable-email-domains/blob/main/disposable_email_blocklist.conf

In [50]:
import requests

url = "https://raw.githubusercontent.com/disposable-email-domains/disposable-email-domains/main/disposable_email_blocklist.conf"
response = requests.get(url)

disposable_domains = {
    line.strip().lower()
    for line in response.text.splitlines()
    if line.strip() and not line.startswith('#')
}


In [51]:
def is_disposable_email(email):
    try:
        domain = email.split('@')[1].lower()
        return domain in disposable_domains
    except:
        return False  # Safe fallback


In [52]:
df3['IsDisposableNew'] = df3['DummyEmails'].apply(is_disposable_email)

In [53]:
disposable_df_new = df3[df3['IsDisposableNew'] == True]
genuine_df_new = df3[df3['IsDisposableNew'] == False]

In [54]:
disposable_df_new

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Completion Day,DummyEmails,IsDisposable,IsDisposableNew
0,Introduction to Artificial Intelligence,MDL-211,75.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:34:07+00:00,2024-08-16,adit.dhiman@trashmail.com,True,True
1,"Climate, Biodiversity, and Human Society (brou...",MDL-285,180.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:03:52+00:00,2024-08-28,adit.dhiman@guerrillamail.com,True,True
6,Make an Impact with Future Technology,MDL-292,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 15:01:14+00:00,2024-08-28,adit.dhiman@guerrillamail.com,True,True
19,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ayesha Maryam,ayxxxx@gmail.com,3088998REG,Completed,2024-08-16 13:18:10+00:00,2024-08-16,ayesha.maryam@mailinator.com,True,True
20,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Ayesha Maryam,ayxxxx@gmail.com,3088998REG,Completed,2024-08-17 08:11:53+00:00,2024-08-17,ayesha.maryam@mailinator.com,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16354,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Vanama mansi Chowdary,NaN,3671281REG,Completed,2024-12-02 06:28:11+00:00,2024-12-02,vanama.mansi.chowdary@trashmail.com,True,True
16356,Create a Credly account,URL-0E39749E2965,5.0,Guidance,Your Learning Builder - Activities,Vanama mansi Chowdary,NaN,3671281REG,Completed,NaT,NaT,vanama.mansi.chowdary@guerrillamail.com,True,True
16364,Build Your First Chatbot,MDL-510,60.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-11-29 18:15:36+00:00,2024-11-29,mamidimada.srikanth@trashmail.com,True,True
16366,Getting Started with Artificial Intelligence,PLAN-E624C2604060,0.0,Learning Plan,Your Learning Builder - Plans,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT,mamidimada.srikanth@mailinator.com,True,True


In [55]:
genuine_df_new

,Learning activity - Title,Learning activity - ID,Learning activity - Duration,Delivery Type,Learning Source Name,Learner - Name,Learner - Email,Learner - ID,Transcript status,Completion Date,Completion Day,DummyEmails,IsDisposable,IsDisposableNew
2,Beyond Conservation to Sustainability,MDL-288,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 13:45:44+00:00,2024-08-28,adit.dhiman@outlook.com,False,False
3,Make an Impact with Data Analytics,MDL-289,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:00:09+00:00,2024-08-28,adit.dhiman@hotmail.com,False,False
4,Make an Impact with AI,MDL-290,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:24:03+00:00,2024-08-28,adit.dhiman@gmail.com,False,False
5,Make an Impact with Hybrid Cloud,MDL-291,90.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-28 14:49:51+00:00,2024-08-28,adit.dhiman@gmail.com,False,False
7,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Adit Dhiman,adxxx@gmail.com,3088627REG,Completed,2024-08-16 11:49:56+00:00,2024-08-16,adit.dhiman@hotmail.com,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16363,Large Language Model Basics,MDL-433,30.0,eLearning,Moodle,Mamidimada Srikanth,NaN,3671287REG,Completed,2024-12-09 15:01:03+00:00,2024-12-09,mamidimada.srikanth@hotmail.com,False,False
16367,Create a Credly account,URL-0E39749E2965,6.0,Guidance,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT,mamidimada.srikanth@hotmail.com,False,False
16368,Earn it! Accept it! Share it! | IBM SkillsBuild,URL-CC7432BB7A8A,2.0,Video,Your Learning Builder - Activities,Mamidimada Srikanth,NaN,3671287REG,Completed,NaT,NaT,mamidimada.srikanth@gmail.com,False,False
16380,Mastering the Art of Prompting,MDL-298,60.0,eLearning,Moodle,Ekalavya sunvith Chichadi,NaN,3671308REG,Completed,2024-11-30 05:38:16+00:00,2024-11-30,ekalavya.sunvith.chichadi@hotmail.com,False,False


### Method 3: Using Kickbox API

In [48]:
import requests

def check_kickbox_disposable(email):
    try:
        url = f"https://open.kickbox.com/v1/disposable/{email}"
        response = requests.get(url)
        data = response.json()
        return data.get("disposable", False)  # True = disposable email
    except Exception as e:
        print(f"Error checking email {email}: {e}")
        return None


In [ ]:
df3['IsDisposableAPI'] = df3['DummyEmails'].apply(check_kickbox_disposable)

In [ ]:
disposable_df_API = df3[df3['IsDisposableAPI'] == True]
genuine_df_API = df3[df3['IsDisposableAPI'] == False]


In [ ]:
disposable_df_API

In [ ]:
genuine_df_API